In [ ]:
%pip install simpy

In [3]:
import simpy
import random
import statistics
from collections import deque
import pandas as pd
import datetime
import numpy as np

In [218]:
# Params and constants. In minutes

#Global Simpy env variables
SEED = 1234
SIM_TIME = 30  # 8 hours of operation #can add a scalar like .5 for testing with a shorter time and to preserve the times in dict below

#Variables for project analysis
HEADWAY = 5 # Cyclical minutes between each train's arrival. Constant set by monorail's operators
DWELL_TIME = 3 # Minutes that a train remains in the station to allow for passenger loading. Constant set by monorail's operators
CAR_CAPACITY = 10 # Number of passengers that the park's model of trains car can hold.
CARS_PER_TRAIN = 3 # Number of train cars coupled together

# Schedule to define peak times throughout park open hours, in 30 min intervals
# Assume empiral data collection determined a peak in the morning at opening time, then again around lunch when people leave and come back. It tapers out like a tail to end the day
arrival_schedule = {1: 15, 2: 12, 3: 10, 4: 5, # First two hours, 8AM to 10AM
                    5: 3, 6: 4, 7: 3, 8: 6, # Next two hours, 10AM to 12PM
                    9: 25, 10: 22, 11: 14, 12: 8, # Next two hours, 12PM to 2PM
                    13: 6, 14: 4, 15: 3 , 16: 1# Final two hours, 2PM to4PM
                    }


In [233]:
def passenger_arrivals(env, store, headway, dwell_time):
    """ Simpy function to generate passenger arrivals at the station platform and add to the FIFO queue"""
    pax_counter = 0

    
    while True:
        #Determine which interval the clock is in, 30 min intervals
        interval = (env.now // 30) + 1
        pax_interarrival = random.expovariate(arrival_schedule[interval]*.05) #remove .2, just slowing down for testing
        yield env.timeout(pax_interarrival)
        pax_counter += 1

        store.put(pax_counter)
        
        pax_queue.append(pax_counter)
        pax_id_list.append(pax_counter)
        pax_arrival_times.append(env.now)
        train_at_creation.append(((env.now - dwell_time) // (headway)) + 1)
        
        
        
        
        mins, seconds = divmod(env.now * 60, 60) #this time format only for printing

        #Debug | Report View
        #print(f"Passenger {pax_counter} arrived at t={int(mins)}:{int(seconds):02d} mins. {((env.now - dwell_time) // (headway)) + 1} ")


        

In [232]:
def train_arrival(env, store, headway=HEADWAY, capacity=CAR_CAPACITY, n_cars=CARS_PER_TRAIN, dwell_time=DWELL_TIME):
    """ Simpy function to create the train arrivals at determined cadence"""
    total_capacity = capacity * n_cars
    train_counter = 0
    
    while True:
        
        yield env.timeout(HEADWAY)
        train_counter += 1

        #Jump into loading process function
        env.process(pax_loading(env, total_capacity, dwell_time, train_counter, store))
        
        mins, seconds = divmod(env.now * 60, 60)  #this time format only for printing
        print(f"=={n_cars} Car monorail with capacity {total_capacity} pax and ID {train_counter} arrived at t={int(mins)}:{int(seconds):02d} mins==")

In [275]:
def pax_loading(env, total_capacity, dwell_time, train_counter, store):
    """ Will remove people from the queue and load the train at a stochastic rate.
    Right now i have a uniform dist for testing values. we might want to change this to something more sophisticated
    """
    # Counter to keep track of pax loaded with each call of this function
    loaded = 0

    # # Helpful output
    # if len(pax_queue) != 0: #mind the case when a 0 len queue gives an index error
    #     print(f"ID starting platform queue = {pax_queue[0]}")
    # elif len(pax_queue) == 0:
    #     print(f"Waiting for passengers on platform!")
        
    print(f"==Monorail dwells and loads for max of {dwell_time} mins==")

    #Elapsed times variables
    train_arrival_time = env.now
    train_departure_time = float(env.now + dwell_time)

    # Repeats until dwell reached
    while env.now < train_departure_time:        
        # Condition for max capacity. ensures no more depart system
        if loaded == total_capacity:
            print(f"==Monorail capacity reached! Queue n={len(pax_queue)}. Sitting then leaving station in t={train_departure_time - env.now:.2f} mins==")
            yield env.timeout(train_departure_time - env.now) #Reaching capacity before dwell time, train must sit until set departure time

        # Condition for people up to capacity
        elif loaded < total_capacity:
            load_time = random.triangular(0, .5, .25)
            if env.now + load_time > train_departure_time:
                env.timeout(train_departure_time - env.now)
            else:
                #Waits for arrival if platform clear, or grabs from queue
                on_platform = store.get()
                #End time
                dwell_end = env.timeout(train_departure_time - env.now)
            
                condition = yield on_platform | dwell_end  
    
                if on_platform in condition:
                    yield env.timeout(load_time)
                    loaded += 1
                    #Debug | Report View
                    #print(f"Loaded passenger n={loaded} @ {env.now:.2f}")            
                    pax_queue.popleft()
                    
                    pax_departure_times.append(env.now)
                    train_at_departure.append(train_counter)
                    queue_len_snapshot.append(len(pax_queue))        
    
    print(f"==Dwell reached for train {train_counter}. Loaded {loaded} pax. Monorail departs at t={env.now}==")
    if len(pax_queue) != 0:
        print(f"Queue n={len(pax_queue)}. ID ending platform queue = {pax_queue[0]}")
    elif len(pax_queue) == 0:
        print(f"Platform cleared! Queue n={len(pax_queue)}.")

In [283]:
#Stat collection data structures
pax_id_list = []
pax_arrival_times = []
pax_departure_times = []
train_at_departure = []
queue_len_snapshot = []
train_at_creation = []

pax_queue = deque() # FIFO queue to manage people arriving at and leaving in order. Stores pax id and arrival timestamp. Eg (id, time) or (4, 1.32)

#random.seed(SEED)
#or
seed = random.seed(datetime.datetime.now().timestamp())
print(seed)

env = simpy.Environment()
pax_store = simpy.Store(env)


#Processes
env.process(passenger_arrivals(env, pax_store, HEADWAY, DWELL_TIME))
env.process(train_arrival(env, pax_store))


env.run(until=SIM_TIME)

#Stat collection

None
==3 Car monorail with capacity 30 pax and ID 1 arrived at t=5:00 mins==
==Monorail dwells and loads for max of 3 mins==
==Dwell reached for train 1. Loaded 8 pax. Monorail departs at t=8.0==
Platform cleared! Queue n=0.
==3 Car monorail with capacity 30 pax and ID 2 arrived at t=10:00 mins==
==Monorail dwells and loads for max of 3 mins==
==Dwell reached for train 2. Loaded 7 pax. Monorail departs at t=13.234601440023045==
Queue n=2. ID ending platform queue = 16
==3 Car monorail with capacity 30 pax and ID 3 arrived at t=15:00 mins==
==Monorail dwells and loads for max of 3 mins==
==Dwell reached for train 3. Loaded 5 pax. Monorail departs at t=18.0==
Queue n=1. ID ending platform queue = 21
==3 Car monorail with capacity 30 pax and ID 4 arrived at t=20:00 mins==
==Monorail dwells and loads for max of 3 mins==
==Dwell reached for train 4. Loaded 2 pax. Monorail departs at t=23.0==
Queue n=2. ID ending platform queue = 23
==3 Car monorail with capacity 30 pax and ID 5 arrived at t

In [223]:
print(pax_queue)
print(len(pax_queue)) #Number left waiting at simlation end time
#should match this
print(len(pax_arrival_times)-len(pax_departure_times))

deque([9, 10, 11, 12, 13, 14])
6
6


In [222]:
print(pax_id_list)
print(pax_arrival_times)
print(pax_departure_times)
#print(n_trains_waited)
print(train_at_creation)

print(len(pax_id_list))
print(len(pax_arrival_times))
print(len(pax_departure_times))
print(len(n_trains_waited))


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[0.4149500525694198, 1.2613698647879026, 4.24283529480248, 7.292144569050798, 10.361904368708649, 10.52886544523641, 15.777584606590903, 18.89764681380569, 19.91825999522085, 22.17068031153575, 22.611198777438126, 26.28117259529562, 28.52984538492508, 28.559008464700163]
[5.231696181521539, 5.371413766881676, 5.468734767382793, 7.706235939387479, 10.835700363099482, 20.172300341158916, 22.325871067140557, 22.807915372946198]
[0.0, 0.0, 1.0, 1.0, 2.0, 2.0, 3.0, 4.0, 4.0, 4.0, 4.0, 5.0, 6.0, 6.0]
14
14
8
142


In [284]:


time_data = {
    'pax_id': pax_id_list,
    'pax_arrival_time': pax_arrival_times,
    'pax_departure_time': pax_departure_times,
    'train_at_creation': train_at_creation,
    'train_at_departure': train_at_departure,
    'queue_len_snapshot': queue_len_snapshot
}
df = pd.DataFrame.from_dict((time_data), orient='index').T
display(df)

,pax_id,pax_arrival_time,pax_departure_time,train_at_creation,train_at_departure,queue_len_snapshot
0,1.0,1.806619,5.287081,0.0,1.0,3.0
1,2.0,1.828956,5.672746,0.0,1.0,3.0
2,3.0,2.838322,5.977980,0.0,1.0,2.0
3,4.0,3.821184,6.110566,1.0,1.0,1.0
4,5.0,5.650311,6.341528,1.0,1.0,0.0
5,6.0,6.447734,6.718613,1.0,1.0,1.0
6,7.0,6.705189,6.891495,1.0,1.0,1.0
7,8.0,6.786369,7.075252,1.0,1.0,0.0
8,9.0,9.472229,10.284580,2.0,2.0,2.0
9,10.0,9.996821,10.718228,2.0,2.0,1.0
